In [2]:
import cv2
from deepface import DeepFace

# เปิดกล้อง
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    try:
        # วิเคราะห์เฟรม (พร้อมระบุ backend ที่เสถียร และป้องกัน crash)
        result = DeepFace.analyze(
            frame,
            actions=['age', 'gender', 'emotion'],
            detector_backend='opencv',  # หรือ 'mediapipe'
            enforce_detection=False
        )

        # ตรวจว่า result เป็น list หรือ dict
        if isinstance(result, list):
            result = result[0]

        age = result.get('age', 'N/A')
        gender = result.get('gender', 'N/A')
        emotion = result.get('dominant_emotion', 'N/A')

        # วาดข้อความบนภาพ
        text = f"Age: {age} | Gender: {gender} | Emotion: {emotion}"
        cv2.putText(frame, text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX,
                    0.9, (0, 255, 0), 2, cv2.LINE_AA)

    except Exception as e:
        print("Error:", e)

    # แสดงภาพ
    cv2.imshow("Real-Time Emotion/Age/Gender Analysis", frame)

    # ออกจากโปรแกรมเมื่อกด 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ปิดกล้องและหน้าต่าง
cap.release()
cv2.destroyAllWindows()

Action: emotion: 100%|██████████| 3/3 [00:00<00:00, 15.01it/s]


In [ ]:
import cv2
from deepface import DeepFace
import os
import numpy as np

# ระบุพาธของไฟล์วิดีโอ
video_path = "girl.mp4"  # เปลี่ยนเป็นชื่อไฟล์วิดีโอของคุณ

# ตรวจสอบว่าไฟล์วิดีโอมีอยู่จริง
if not os.path.exists(video_path):
    print(f"ไม่พบไฟล์วิดีโอ: {video_path}")
    print("กรุณาใส่ชื่อไฟล์วิดีโอที่ถูกต้อง")
    exit()

# เปิดไฟล์วิดีโอ
cap = cv2.VideoCapture(video_path)

# ตรวจสอบว่าเปิดวิดีโอได้หรือไม่
if not cap.isOpened():
    print("ไม่สามารถเปิดไฟล์วิดีโอได้")
    exit()

# ดึงข้อมูลวิดีโอ
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = total_frames / fps

print(f"กำลังวิเคราะห์วิดีโอ: {video_path}")
print(f"FPS: {fps}, จำนวนเฟรม: {total_frames}, ระยะเวลา: {duration:.2f} วินาที")

frame_count = 0

# เก็บข้อมูลการวิเคราะห์ล่าสุด (สำหรับตาราง)
analysis_history = []
max_history = 10  # แสดงข้อมูล 10 ช่วงเวลาล่าสุด

def create_table_image(history, current_data=None):
    """สร้างภาพตารางข้อมูลการวิเคราะห์"""
    table_height = 400
    table_width = 600
    table_img = np.zeros((table_height, table_width, 3), dtype=np.uint8)
    table_img.fill(30)  # พื้นหลังสีเทาเข้ม
    
    # หัวตาราง
    headers = ["Time(s)", "Age", "Gender", "Emotion"]
    header_y = 40
    col_widths = [100, 80, 120, 200]
    col_x = [50, 150, 230, 350]
    
    # วาดหัวตาราง
    for i, header in enumerate(headers):
        cv2.putText(table_img, header, (col_x[i], header_y), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    # วาดเส้นใต้หัวตาราง
    cv2.line(table_img, (30, header_y + 10), (table_width - 30, header_y + 10), 
             (255, 255, 255), 2)
    
    # วาดข้อมูลในตาราง
    start_y = header_y + 40
    row_height = 30
    
    # แสดงข้อมูลปัจจุบัน (สีเขียว)
    if current_data:
        y_pos = start_y
        data = [
            f"{current_data['time']:.1f}",
            str(current_data['age']),
            current_data['gender'],
            current_data['emotion']
        ]
        
        for i, value in enumerate(data):
            cv2.putText(table_img, value, (col_x[i], y_pos), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        
        # วาดเส้นใต้แถวปัจจุบัน
        cv2.line(table_img, (30, y_pos + 5), (table_width - 30, y_pos + 5), 
                 (0, 255, 0), 1)
    
    # แสดงประวัติ (สีขาว)
    for idx, record in enumerate(history[-9:]):  # แสดง 9 แถวก่อนหน้า
        y_pos = start_y + (idx + 2) * row_height
        if y_pos > table_height - 30:
            break
            
        data = [
            f"{record['time']:.1f}",
            str(record['age']),
            record['gender'],
            record['emotion']
        ]
        
        for i, value in enumerate(data):
            cv2.putText(table_img, value, (col_x[i], y_pos), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, (200, 200, 200), 1)
    
    # วาดกรอบตาราง
    cv2.rectangle(table_img, (30, 20), (table_width - 30, table_height - 20), 
                  (255, 255, 255), 2)
    
    # วาดเส้นแนวตั้ง
    for x in col_x[1:]:
        cv2.line(table_img, (x - 10, 20), (x - 10, table_height - 20), 
                 (255, 255, 255), 1)
    
    return table_img

while True:
    ret, frame = cap.read()
    if not ret:
        print("จบการวิเคราะห์วิดีโอ")
        break

    frame_count += 1
    current_time = frame_count / fps

    try:
        # วิเคราะห์เฟรม (ทุก 5 เฟรมเพื่อลดการประมวลผล)
        if frame_count % 5 == 0:
            result = DeepFace.analyze(
                frame,
                actions=['age', 'gender', 'emotion'],
                detector_backend='opencv',
                enforce_detection=False
            )

            # ตรวจว่า result เป็น list หรือ dict
            if isinstance(result, list):
                result = result[0]

            age = result.get('age', 'N/A')
            gender_data = result.get('gender', {})
            if isinstance(gender_data, dict):
                gender = max(gender_data, key=gender_data.get) if gender_data else 'N/A'
            else:
                gender = gender_data
            emotion = result.get('dominant_emotion', 'N/A')

            # เก็บข้อมูลล่าสุด
            current_analysis = {
                'time': current_time,
                'age': age,
                'gender': gender,
                'emotion': emotion
            }
            
            # เพิ่มข้อมูลใหม่ลงในประวัติ
            analysis_history.append(current_analysis.copy())
            if len(analysis_history) > max_history:
                analysis_history.pop(0)
        else:
            # ใช้ข้อมูลล่าสุดถ้าไม่ได้วิเคราะห์เฟรมนี้
            if analysis_history:
                current_analysis = analysis_history[-1].copy()
                current_analysis['time'] = current_time
            else:
                current_analysis = {
                    'time': current_time,
                    'age': 'N/A',
                    'gender': 'N/A', 
                    'emotion': 'N/A'
                }

        # วาดข้อความบนวิดีโอ
        text1 = f"Age: {current_analysis['age']} | Gender: {current_analysis['gender']} | Emotion: {current_analysis['emotion']}"
        text2 = f"Time: {current_time:.2f}s | Frame: {frame_count}/{total_frames}"
        
        cv2.putText(frame, text1, (20, 40), cv2.FONT_HERSHEY_SIMPLEX,
                    0.8, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, text2, (20, 80), cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (255, 255, 255), 2, cv2.LINE_AA)

        # สร้างตารางข้อมูล
        table_img = create_table_image(analysis_history[:-1] if len(analysis_history) > 1 else [], 
                                     current_analysis)

        # แสดงความคืบหน้า
        progress = (frame_count / total_frames) * 100
        print(f"\rกำลังประมวลผล: {progress:.1f}% ({frame_count}/{total_frames})", end="")

    except Exception as e:
        print(f"\nError ที่เฟรม {frame_count}: {e}")
        # สร้างตารางว่างถ้าเกิดข้อผิดพลาด
        table_img = create_table_image(analysis_history)

    # แสดงภาพวิดีโอและตาราง
    cv2.imshow("Video Analysis", frame)
    cv2.imshow("Analysis Data Table", table_img)

    # ออกจากโปรแกรมเมื่อกด 'q' หรือ 'ESC'
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q') or key == 27:  # 'q' หรือ ESC
        print("\nหยุดการวิเคราะห์โดยผู้ใช้")
        break

# ปิดวิดีโอและหน้าต่าง
cap.release()
cv2.destroyAllWindows()
print("\nเสร็จสิ้นการวิเคราะห์")

: 

In [5]:
import cv2
from deepface import DeepFace
import os
import numpy as np
import mediapipe as mp
import tensorflow as tf

# ตั้งค่าให้ใช้ GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("✓ GPU พร้อมใช้งาน")
else:
    print("⚠ ไม่พบ GPU - ใช้ CPU")

# ตั้งค่า MediaPipe สำหรับ Face Landmarks
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# สร้าง Face Mesh detector
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# ระบุพาธของไฟล์วิดีโอ
video_path = "girl.mp4"  # เปลี่ยนเป็นชื่อไฟล์วิดีโอของคุณ

# ตรวจสอบว่าไฟล์วิดีโอมีอยู่จริง
if not os.path.exists(video_path):
    print(f"ไม่พบไฟล์วิดีโอ: {video_path}")
    print("กรุณาใส่ชื่อไฟล์วิดีโอที่ถูกต้อง")
    exit()

# เปิดไฟล์วิดีโอ
cap = cv2.VideoCapture(video_path)

# ตรวจสอบว่าเปิดวิดีโอได้หรือไม่
if not cap.isOpened():
    print("ไม่สามารถเปิดไฟล์วิดีโอได้")
    exit()

# ดึงข้อมูลวิดีโอ
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = total_frames / fps

print(f"กำลังวิเคราะห์วิดีโอ: {video_path}")
print(f"FPS: {fps}, จำนวนเฟรม: {total_frames}, ระยะเวลา: {duration:.2f} วินาที")

frame_count = 0

# เก็บข้อมูลการวิเคราะห์ล่าสุด (สำหรับตาราง)
analysis_history = []
max_history = 10  # แสดงข้อมูล 10 ช่วงเวลาล่าสุด

def draw_face_landmarks(image, landmarks, connections=None):
    """วาดจุดมาร์คบนใบหน้า"""
    if landmarks is None:
        return image
    
    h, w, _ = image.shape
    
    # วาดจุดสำคัญ (ดวงตา, จมูก, ปาก, คิ้ว)
    important_points = [
        # ดวงตาซ้าย
        33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246,
        # ดวงตาขวา  
        362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398,
        # คิ้วซ้าย
        46, 53, 52, 51, 48, 115, 131, 134, 102, 49, 220, 305,
        # คิ้วขวา
        276, 283, 282, 295, 285, 336, 296, 334, 293, 300, 276, 283,
        # จมูก
        1, 2, 5, 4, 6, 19, 20, 94, 125, 141, 235, 236, 3, 51, 48, 115, 131, 134, 102, 49, 220, 305,
        # ปาก
        0, 17, 18, 200, 199, 175, 0, 16, 17, 18, 200, 199, 175, 8, 9, 10, 151, 8, 9, 10, 151
    ]
    
    # วาดจุดหลัก
    for idx in important_points:
        if idx < len(landmarks.landmark):
            x = int(landmarks.landmark[idx].x * w)
            y = int(landmarks.landmark[idx].y * h)
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
    
    # วาดเส้นเชื่อมจุดสำคัญ
    # ดวงตาซ้าย
    left_eye = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246]
    for i in range(len(left_eye)):
        if left_eye[i] < len(landmarks.landmark) and left_eye[(i+1)%len(left_eye)] < len(landmarks.landmark):
            x1 = int(landmarks.landmark[left_eye[i]].x * w)
            y1 = int(landmarks.landmark[left_eye[i]].y * h)
            x2 = int(landmarks.landmark[left_eye[(i+1)%len(left_eye)]].x * w)
            y2 = int(landmarks.landmark[left_eye[(i+1)%len(left_eye)]].y * h)
            cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 1)
    
    # ดวงตาขวา
    right_eye = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
    for i in range(len(right_eye)):
        if right_eye[i] < len(landmarks.landmark) and right_eye[(i+1)%len(right_eye)] < len(landmarks.landmark):
            x1 = int(landmarks.landmark[right_eye[i]].x * w)
            y1 = int(landmarks.landmark[right_eye[i]].y * h)
            x2 = int(landmarks.landmark[right_eye[(i+1)%len(right_eye)]].x * w)
            y2 = int(landmarks.landmark[right_eye[(i+1)%len(right_eye)]].y * h)
            cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 1)
    
    # เส้นขอบใบหน้า
    face_oval = [10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103, 67, 109]
    for i in range(len(face_oval)):
        if face_oval[i] < len(landmarks.landmark) and face_oval[(i+1)%len(face_oval)] < len(landmarks.landmark):
            x1 = int(landmarks.landmark[face_oval[i]].x * w)
            y1 = int(landmarks.landmark[face_oval[i]].y * h)
            x2 = int(landmarks.landmark[face_oval[(i+1)%len(face_oval)]].x * w)
            y2 = int(landmarks.landmark[face_oval[(i+1)%len(face_oval)]].y * h)
            cv2.line(image, (x1, y1), (x2, y2), (0, 255, 255), 1)
    
    return image

def create_table_image(history, current_data=None):
    """สร้างภาพตารางข้อมูลการวิเคราะห์"""
    table_height = 400
    table_width = 650
    table_img = np.zeros((table_height, table_width, 3), dtype=np.uint8)
    table_img.fill(30)  # พื้นหลังสีเทาเข้ม
    
    # หัวตาราง
    headers = ["Time(s)", "Age", "Gender", "Emotion", "GPU"]
    header_y = 40
    col_widths = [80, 60, 100, 150, 80]
    col_x = [30, 110, 170, 270, 420]
    
    # วาดหัวตาราง
    for i, header in enumerate(headers):
        cv2.putText(table_img, header, (col_x[i], header_y), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
    # วาดเส้นใต้หัวตาราง
    cv2.line(table_img, (20, header_y + 10), (table_width - 20, header_y + 10), 
             (255, 255, 255), 2)
    
    # วาดข้อมูลในตาราง
    start_y = header_y + 40
    row_height = 30
    
    # แสดงข้อมูลปัจจุบัน (สีเขียว)
    if current_data:
        y_pos = start_y
        gpu_status = "ON" if len(physical_devices) > 0 else "OFF"
        data = [
            f"{current_data['time']:.1f}",
            str(current_data['age']),
            current_data['gender'],
            current_data['emotion'],
            gpu_status
        ]
        
        for i, value in enumerate(data):
            color = (0, 255, 0) if i < 4 else (0, 255, 255) if gpu_status == "ON" else (0, 0, 255)
            cv2.putText(table_img, value, (col_x[i], y_pos), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
        
        # วาดเส้นใต้แถวปัจจุบัน
        cv2.line(table_img, (20, y_pos + 5), (table_width - 20, y_pos + 5), 
                 (0, 255, 0), 1)
    
    # แสดงประวัติ (สีขาว)
    for idx, record in enumerate(history[-8:]):  # แสดง 8 แถวก่อนหน้า
        y_pos = start_y + (idx + 2) * row_height
        if y_pos > table_height - 30:
            break
            
        gpu_status = "ON" if len(physical_devices) > 0 else "OFF"
        data = [
            f"{record['time']:.1f}",
            str(record['age']),
            record['gender'],
            record['emotion'],
            gpu_status
        ]
        
        for i, value in enumerate(data):
            cv2.putText(table_img, value, (col_x[i], y_pos), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.35, (180, 180, 180), 1)
    
    # วาดกรอบตาราง
    cv2.rectangle(table_img, (20, 20), (table_width - 20, table_height - 20), 
                  (255, 255, 255), 2)
    
    # วาดเส้นแนวตั้ง
    for x in col_x[1:]:
        cv2.line(table_img, (x - 10, 20), (x - 10, table_height - 20), 
                 (255, 255, 255), 1)
    
    # แสดงสถานะ GPU ด้านบน
    gpu_text = f"GPU Status: {'ENABLED' if len(physical_devices) > 0 else 'DISABLED'}"
    gpu_color = (0, 255, 0) if len(physical_devices) > 0 else (0, 0, 255)
    cv2.putText(table_img, gpu_text, (table_width - 200, 15), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.4, gpu_color, 1)
    
    return table_img

while True:
    ret, frame = cap.read()
    if not ret:
        print("จบการวิเคราะห์วิดีโอ")
        break

    frame_count += 1
    current_time = frame_count / fps
    
    # แปลงสีสำหรับ MediaPipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    try:
        # ตรวจจับจุดมาร์คบนใบหน้า
        results = face_mesh.process(rgb_frame)
        
        # วาดจุดมาร์คบนใบหน้า
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # วิธีแสดงแบบง่าย (จุดเท่านั้น)
                # mp_drawing.draw_landmarks(
                #     frame, face_landmarks, mp_face_mesh.FACEMESH_CONTOURS,
                #     None, mp_drawing_styles.get_default_face_mesh_contours_style())
                
                # วิธีแสดงแบบกำหนดเอง
                frame = draw_face_landmarks(frame, face_landmarks)
        
        # วิเคราะห์อายุ เพศ อารมณ์ (ทุก 5 เฟรมเพื่อลดการประมวลผล)
        if frame_count % 5 == 0:
            with tf.device('/GPU:0' if len(physical_devices) > 0 else '/CPU:0'):
                result = DeepFace.analyze(
                    frame,
                    actions=['age', 'gender', 'emotion'],
                    detector_backend='opencv',
                    enforce_detection=False
                )

            # ตรวจว่า result เป็น list หรือ dict
            if isinstance(result, list):
                result = result[0]

            age = result.get('age', 'N/A')
            gender_data = result.get('gender', {})
            if isinstance(gender_data, dict):
                gender = max(gender_data, key=gender_data.get) if gender_data else 'N/A'
            else:
                gender = gender_data
            emotion = result.get('dominant_emotion', 'N/A')

            # เก็บข้อมูลล่าสุด
            current_analysis = {
                'time': current_time,
                'age': age,
                'gender': gender,
                'emotion': emotion
            }
            
            # เพิ่มข้อมูลใหม่ลงในประวัติ
            analysis_history.append(current_analysis.copy())
            if len(analysis_history) > max_history:
                analysis_history.pop(0)
        else:
            # ใช้ข้อมูลล่าสุดถ้าไม่ได้วิเคราะห์เฟรมนี้
            if analysis_history:
                current_analysis = analysis_history[-1].copy()
                current_analysis['time'] = current_time
            else:
                current_analysis = {
                    'time': current_time,
                    'age': 'N/A',
                    'gender': 'N/A', 
                    'emotion': 'N/A'
                }

        # วาดข้อความบนวิดีโอ
        gpu_status = "GPU" if len(physical_devices) > 0 else "CPU"
        text1 = f"Age: {current_analysis['age']} | Gender: {current_analysis['gender']} | Emotion: {current_analysis['emotion']}"
        text2 = f"Time: {current_time:.2f}s | Frame: {frame_count}/{total_frames} | Device: {gpu_status}"
        
        cv2.putText(frame, text1, (20, 40), cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, text2, (20, 80), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (255, 255, 255), 2, cv2.LINE_AA)

        # สร้างตารางข้อมูล
        table_img = create_table_image(analysis_history[:-1] if len(analysis_history) > 1 else [], 
                                     current_analysis)

        # แสดงความคืบหน้า
        progress = (frame_count / total_frames) * 100
        print(f"\rกำลังประมวลผล: {progress:.1f}% ({frame_count}/{total_frames}) [{gpu_status}]", end="")

    except Exception as e:
        print(f"\nError ที่เฟรม {frame_count}: {e}")
        # สร้างตารางว่างถ้าเกิดข้อผิดพลาด
        table_img = create_table_image(analysis_history)

    # แสดงภาพวิดีโอและตาราง
    cv2.imshow("Video Analysis with Face Landmarks", frame)
    cv2.imshow("Analysis Data Table", table_img)

    # แสดงคำแนะนำ
    if frame_count == 1:
        print(f"\n{'='*60}")
        print("คำแนะนำการใช้งาน:")
        print("- กด 'q' หรือ 'ESC' เพื่อออกจากโปรแกรม")
        print("- กด 'Space' เพื่อหยุดชั่วคราว")
        print("- สีเขียว: จุดมาร์คบนใบหน้า")
        print("- สีน้ำเงิน: เส้นรอบดวงตา")
        print("- สีเหลือง: เส้นขอบใบหน้า")
        print(f"{'='*60}")

    # ออกจากโปรแกรมเมื่อกด 'q' หรือ 'ESC' หรือ 'Space' เพื่อหยุดชั่วคราว
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q') or key == 27:  # 'q' หรือ ESC
        print("\nหยุดการวิเคราะห์โดยผู้ใช้")
        break
    elif key == ord(' '):  # Space bar
        print("\nหยุดชั่วคราว - กด 'c' เพื่อดำเนินการต่อ หรือ 'q' เพื่อออก")
        while True:
            key = cv2.waitKey(0) & 0xFF
            if key == ord('c'):
                print("ดำเนินการต่อ...")
                break
            elif key == ord('q') or key == 27:
                print("ออกจากโปรแกรม...")
                cap.release()
                cv2.destroyAllWindows()
                exit()

# ปิดวิดีโอและหน้าต่าง
cap.release()
cv2.destroyAllWindows()
print(f"\nเสร็จสิ้นการวิเคราะห์ - ใช้อุปกรณ์: {gpu_status}")

⚠ ไม่พบ GPU - ใช้ CPU
กำลังวิเคราะห์วิดีโอ: girl.mp4
FPS: 23, จำนวนเฟรม: 6664, ระยะเวลา: 289.74 วินาที
กำลังประมวลผล: 0.0% (1/6664) [CPU]
คำแนะนำการใช้งาน:
- กด 'q' หรือ 'ESC' เพื่อออกจากโปรแกรม
- กด 'Space' เพื่อหยุดชั่วคราว
- สีเขียว: จุดมาร์คบนใบหน้า
- สีน้ำเงิน: เส้นรอบดวงตา
- สีเหลือง: เส้นขอบใบหน้า
กำลังประมวลผล: 0.1% (4/6664) [CPU]

Action: emotion: 100%|██████████| 3/3 [00:00<00:00, 13.63it/s]


กำลังประมวลผล: 0.1% (9/6664) [CPU]

Action: emotion: 100%|██████████| 3/3 [00:00<00:00, 14.87it/s]


กำลังประมวลผล: 0.2% (14/6664) [CPU]

...

กำลังประมวลผล: 9.1% (609/6664) [CPU]

Action: emotion: 100%|██████████| 3/3 [00:00<00:00, 16.79it/s]


กำลังประมวลผล: 9.2% (614/6664) [CPU]

Action: emotion: 100%|██████████| 3/3 [00:00<00:00, 17.01it/s]


กำลังประมวลผล: 9.2% (615/6664) [CPU]
หยุดการวิเคราะห์โดยผู้ใช้

เสร็จสิ้นการวิเคราะห์ - ใช้อุปกรณ์: CPU


In [3]:
import tensorflow as tf
print("GPU devices:", tf.config.list_physical_devices('GPU'))

GPU devices: []


In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("GPU available:", tf.test.is_gpu_available())

TensorFlow version: 2.19.0
Built with CUDA: False
GPU available: False


In [4]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

TensorFlow version: 2.19.0
Num GPUs Available: 0
